In [1]:
import sqlite3
con = sqlite3.connect("SQL_4.db")
cur = con.cursor()

In [2]:
cur.execute("DROP TABLE IF EXISTS Movies")
cur.execute("DROP TABLE IF EXISTS MovieTheaters")

In [3]:
cur.execute("""
CREATE TABLE IF NOT EXISTS Movies (
  Code INTEGER,
  Title VARCHAR(255) NOT NULL,
  Rating VARCHAR(255),
  PRIMARY KEY (Code)
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS MovieTheaters (
  Code INTEGER,
  Name VARCHAR(255) NOT NULL,
  Movie INTEGER,
  PRIMARY KEY (Code)
);
""")

In [4]:
cur.executescript("""
INSERT INTO Movies(Code,Title,Rating) VALUES(1,'Citizen Kane','PG');
INSERT INTO Movies(Code,Title,Rating) VALUES(2,'Singin'' in the Rain','G');
INSERT INTO Movies(Code,Title,Rating) VALUES(3,'The Wizard of Oz','G');
INSERT INTO Movies(Code,Title,Rating) VALUES(4,'The Quiet Man',NULL);
INSERT INTO Movies(Code,Title,Rating) VALUES(5,'North by Northwest',NULL);
INSERT INTO Movies(Code,Title,Rating) VALUES(6,'The Last Tango in Paris','NC-17');
INSERT INTO Movies(Code,Title,Rating) VALUES(7,'Some Like it Hot','PG-13');
INSERT INTO Movies(Code,Title,Rating) VALUES(8,'A Night at the Opera',NULL);
 
INSERT INTO MovieTheaters(Code,Name,Movie) VALUES(1,'Odeon',5);
INSERT INTO MovieTheaters(Code,Name,Movie) VALUES(2,'Imperial',1);
INSERT INTO MovieTheaters(Code,Name,Movie) VALUES(3,'Majestic',NULL);
INSERT INTO MovieTheaters(Code,Name,Movie) VALUES(4,'Royale',6);
INSERT INTO MovieTheaters(Code,Name,Movie) VALUES(5,'Paraiso',3);
INSERT INTO MovieTheaters(Code,Name,Movie) VALUES(6,'Nickelodeon',NULL);
""")

In [5]:
def run(cmd):
    r = cur.execute(cmd)
    display(r.fetchall())

## 4.1 Select the title of all movies.

In [6]:
run("""
SELECT Title 
FROM Movies
;""")

[('Citizen Kane',),
 ("Singin' in the Rain",),
 ('The Wizard of Oz',),
 ('The Quiet Man',),
 ('North by Northwest',),
 ('The Last Tango in Paris',),
 ('Some Like it Hot',),
 ('A Night at the Opera',)]

## 4.2 Show all the distinct ratings in the database.

In [7]:
run("""
SELECT DISTINCT(Rating) 
FROM Movies
;""")

[('PG',), ('G',), (None,), ('NC-17',), ('PG-13',)]

## 4.3 Show all unrated movies.

In [8]:
run("""
SELECT Title 
FROM Movies
WHERE Rating ISNULL
;""")

[('The Quiet Man',), ('North by Northwest',), ('A Night at the Opera',)]

## 4.4 Select all movie theaters that are not currently showing a movie.

In [9]:
run("""
SELECT Name 
FROM MovieTheaters
WHERE Movie ISNULL
;""")

[('Majestic',), ('Nickelodeon',)]

## 4.5 Select all data from all movie theaters and, additionally, the data from the movie that is being shown in the theater (if one is being shown).

In [10]:
run("""
SELECT *
FROM MovieTheaters
LEFT JOIN Movies 
ON MovieTheaters.Movie = Movies.Code
;""")

[(1, 'Odeon', 5, 5, 'North by Northwest', None),
 (2, 'Imperial', 1, 1, 'Citizen Kane', 'PG'),
 (3, 'Majestic', None, None, None, None),
 (4, 'Royale', 6, 6, 'The Last Tango in Paris', 'NC-17'),
 (5, 'Paraiso', 3, 3, 'The Wizard of Oz', 'G'),
 (6, 'Nickelodeon', None, None, None, None)]

## 4.6 Select all data from all movies and, if that movie is being shown in a theater, show the data from the theater.

In [11]:
run("""
SELECT *
FROM Movies
LEFT JOIN MovieTheaters 
ON MovieTheaters.Movie = Movies.Code
;""")

[(1, 'Citizen Kane', 'PG', 2, 'Imperial', 1),
 (2, "Singin' in the Rain", 'G', None, None, None),
 (3, 'The Wizard of Oz', 'G', 5, 'Paraiso', 3),
 (4, 'The Quiet Man', None, None, None, None),
 (5, 'North by Northwest', None, 1, 'Odeon', 5),
 (6, 'The Last Tango in Paris', 'NC-17', 4, 'Royale', 6),
 (7, 'Some Like it Hot', 'PG-13', None, None, None),
 (8, 'A Night at the Opera', None, None, None, None)]

## 4.7 Show the titles of movies not currently being shown in any theaters.

In [14]:
run("""
SELECT Title
FROM Movies
LEFT JOIN MovieTheaters
ON MovieTheaters.Movie = Movies.Code
WHERE Movie IS NULL
;""")

[("Singin' in the Rain",),
 ('The Quiet Man',),
 ('Some Like it Hot',),
 ('A Night at the Opera',)]

## 4.8 Add the unrated movie "One, Two, Three".

In [16]:
run("""
INSERT INTO Movies (Code, Title, Rating)
VALUES (9, 'One, Two, Three', Null)
;
""")

[]

## 4.9 Set the rating of all unrated movies to "G".

In [17]:
run("""
UPDATE Movies 
SET Rating = 'G'
WHERE Rating = Null 
;
""")

[]

## 4.10 Remove movie theaters projecting movies rated "NC-17".

In [18]:
cur.execute("""
    DELETE FROM MovieTheaters
    WHERE Code IN (
        SELECT MovieTheaters.Code
        FROM MovieTheaters
        LEFT JOIN Movies ON MovieTheaters.Movie = Movies.Code
        WHERE Movies.Rating = 'NC-17'
    )
""")

In [19]:
con.close()